In [16]:
import unicodedata
import string
import pandas as pd
import re
import numpy as np

import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

all_letters = string.ascii_letters + " .,;'0123456789"
n_letters = len(all_letters)

all_letters

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'0123456789"

Build Category dictionary, category and list of product name, shor description

In [2]:
from elasticsearch import Elasticsearch
client = Elasticsearch("http://172.17.212.35:9200")
resp = client.search(index="retromotion-indexer_development_products",
                     body={"_source":["descriptions","descriptionsSource","nameSource","shortDescriptionSource","categoriesSource"],
                           "query": {"match_all": {}}})

In [4]:
df_eng = pd.DataFrame(columns=['name','sdesc','category','catlevel0','catlevel1','catlevel2','catlevel3','catlevel4','catlevel5'])

In [5]:
def cleanStr(text):
   
    # remove numbers
    # no_number_string = re.sub(r'\d+','',lower_string)

    # remove all punctuation except words and space
    # text = re.sub(r'[^\w\s]','', lower_string)

    # clean = re.compile('<.*?>')

    # text = re.sub(clean,'',text)
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' .join(emoticons).replace('-', ''))

    return text


In [6]:
name_en=[]
name_de=[]

sdesc_en=[]
sdesc_de=[]

desc_en=[]
desc_de=[]

cat_en=[]
cat_de=[]

catpath_en=[]
catpath_de=[]

for hit in resp['hits']['hits']:
    list_row =[]
    for name in hit['_source']['nameSource']:
        if (name["language"]=="en"):
            list_row.append(cleanStr(name["value"]))
        else:
            name_de.append(name["value"])
    
    for sdesc in hit['_source']['shortDescriptionSource']:
        if (sdesc["language"]=="en"):
            list_row.append(cleanStr(sdesc["value"]))
        else:
            sdesc_de.append(sdesc["value"])

    # for desc in hit['_source']['descriptionsSource']:
    #     if (desc["language"]=="en"):
    #         desc_en.append(readHtml(desc["value"]))
    #     else:
    #         desc_de.append(desc["value"])
    
    for cats in hit['_source']['categoriesSource']:
        if (cats["language"]=="en"):
            list_row.append(cleanStr(cats["label"]))
        else:
            desc_de.append(cats["label"])

        if (cats["language"]=="en"):
            categories =cats["path"].split('/')
            i =0
            catls=[]
            for category in categories:

                list_row.append(cleanStr(category))
                i= i+1
            
        else:
            catpath_de.append(cats["path"])
    
    # new_row = pd.DataFrame({'name':_name, 'sdesc':_sdesc, 'category':_cat_en }, index=[0])

    # pd.concat([new_row,df_eng.loc[:]]).reset_index(drop=True)

    for i in range (0,9-len(list_row)):
        list_row.append(np.nan)

    df_eng.loc[len(df_eng)] = list_row



In [7]:
df_eng.head()

,name,sdesc,category,catlevel0,catlevel1,catlevel2,catlevel3,catlevel4,catlevel5
0,denso dtm82363 thermostat coolant,thermostat coolant,thermostat,sparepart,cooling system,thermostat,NaN,NaN,NaN
1,zf 1043 010 297 automatic transmission,automatic transmission,automatic transmission,sparepart,transmission,automatic transmission,automatic transmission,NaN,NaN
2,zf 1060 040 003 automatic transmission,automatic transmission,automatic transmission,sparepart,transmission,automatic transmission,automatic transmission,NaN,NaN
3,vaico v10 0131 camshaft,camshaft,camshaft,sparepart,engine,engine timing,camshaft,NaN,NaN
4,vaico v26 0302 track control arm,track control arm,track control arm,sparepart,powertrain suspension,wheel suspension,track control arm,NaN,NaN


In [54]:
df_category = df_eng.groupby('category')

In [46]:
category.get_group('automatic transmission')['name']



1    zf 1043 010 297 automatic transmission
2    zf 1060 040 003 automatic transmission
Name: name, dtype: object

In [55]:
# extract keys from groups
keys = df_category.groups.keys()
# for i in keys:
#     print(category.get_group(i))
#     print('\n')

    # Display result

all_categories=list(keys)

Result:
 dict_keys(['automatic transmission', 'camshaft', 'mounting', 'shock absorber', 'suspension kit', 'tensioner pulley', 'thermostat', 'track control arm', 'wheel hub'])


#### Turning product Names into Tensors
        	    
Represent each name in one-hot encoded form (1 for the position of the letter, 0 elsewhere)

To make a word we join a bunch of those into a 2D matrix <line_length x 1 x n_letters>.
That extra 1 dimension is because PyTorch assumes everything is in batches - we're just using a batch size of 1 here.

In [17]:
import torch

In [18]:
def letterToTensor(letter):
    
    tensor = torch.zeros(1, n_letters)
    tensor[0][all_letters.find(letter)] = 1
    
    return tensor

In [19]:
print(letterToTensor('a'))

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [20]:
def nameToTensor(name):
    tensor = torch.zeros(len(name), 1, n_letters)
    
    for li, letter in enumerate(name):
        tensor[li][0][all_letters.find(letter)] = 1
    
    return tensor

In [21]:
p_tensor = nameToTensor('denso dtm82363 thermostat coolant')

p_tensor.size()

torch.Size([33, 1, 67])

In [22]:
p_tensor

tensor([[[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.]]])

In [23]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        
        hidden = self.i2h(combined)
        
        output = self.i2o(combined)
        output = self.softmax(output)
        
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [28]:
n_hidden = 256
n_category = len(category)

rnn = RNN(n_letters, n_hidden, n_category)

In [29]:
inp = letterToTensor('C')

hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(inp, hidden)

print('output size =', output.size())
print('next_hidden size =', next_hidden.size())

output size = torch.Size([1, 9])
next_hidden size = torch.Size([1, 256])


In [30]:
inp = nameToTensor('denso dtm82363 thermostat coolant')

hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(inp[0], hidden)

In [31]:
print(output)

tensor([[-2.2812, -2.2042, -2.2270, -2.1712, -2.1616, -2.1168, -2.2543, -2.1659,
         -2.2031]], grad_fn=<LogSoftmaxBackward0>)


In [44]:
def categoryFromOutput(output):
    
    _, top_i = output.topk(1)
    
    cat_i = top_i[0].item()
    
    return all_categories[cat_i], cat_i

In [45]:
print(categoryFromOutput(output))

('tensioner pulley', 5)


#### get training samples category and names

In [51]:
import random

def randomTrainingExample():
    
    random_cat_index = random.randint(0, n_category - 1)
    randcategory = all_categories[random_cat_index]
    
    # get feature name from the category
    random_feature_names = category.get_group(randcategory)['name']
    
    name = random_feature_names[random.randint(0, len(random_feature_names) - 1)]
    
    category_tensor = torch.tensor([all_categories.index(randcategory)], dtype=torch.long)
    name_tensor = nameToTensor(name)
    
    return randcategory, name, category_tensor, name_tensor

In [53]:
for i in range(10):
    category, name, category_tensor, name_tensor = randomTrainingExample()
    
    print('category =', category, ', name =', name)

AttributeError: 'str' object has no attribute 'get_group'